In [1]:
import torch
import numpy as np
import time
import music_data_utils
from crnngan import CRNNGAN
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Get the data loader
musicDataLoader = music_data_utils.MusicDataLoader('./music_data', 0, 0)

Data loader: datadir: ./music_data
num genres:1
num composers: 154
limit works per composer: None
Reading files classical/adam: 3
Reading files classical/aguado: 5
Reading files classical/albenizisaac: 20
Reading files classical/albenizmateo: 2
Reading files classical/albinoni: 1
Reading files classical/alford: 2
Reading files classical/alkan: 11
Reading files classical/anderson: 8
Reading files classical/ansell: 1
Reading files classical/arensky: 5
Reading files classical/arriaga: 2
Reading files classical/bach: 100
Reading files classical/bach: 200
Reading files classical/bach: 300
Reading files classical/bach: 360
Reading files classical/barber: 8
Path does not exist: ./music_data/classical/barbieri
Reading files classical/bartok: 99
Reading files classical/bax: 2
Error reading ./music_data/classical/beethoven/14715thSympnmov1.mid
Reading files classical/beethoven: 68
Reading files classical/bellini: 4
Reading files classical/bergmuller: 48
Reading files classical/berlin: 3
Reading 

In [7]:
batch_length = 10
sequence_length = 30
_, data = musicDataLoader.get_batch(batch_length, sequence_length)
data.shape

(10, 30, 4)

In [3]:
min_batch = []
max_batch = []
batch_size = 200
seq_length = 200
musicDataLoader.rewind('train')
_, batch_data = musicDataLoader.get_batch(batch_size, seq_length, 'train')
while batch_data is not None:
    min_batch.append([batch_data.min(axis=0)])
    max_batch.append([batch_data.max(axis=0)])
    _, batch_data = musicDataLoader.get_batch(batch_size, seq_length, 'train')
    
min_batch = np.array(min_batch)
min_total = min_batch.reshape(-1, 4).min(axis=0)
max_batch = np.array(max_batch)
max_total = max_batch.reshape(-1, 4).max(axis=0)
print('Min: ', min_total, ', max: ', max_total)

Min:  [0. 0. 0. 0.] , max:  [ 6931.2        60576.         12543.85395142   127.        ]


In [4]:
batch_length = 100
sequence_length = 30
in_channels_g = 20
out_channels_g = 4
hidden_dim_g = 350
### In the paper they indicate they've done it with 2, but it does not seem enough to get the complexity
### of the data.
n_layers_g = 3
in_channels_d = 4
out_channels_d = 2
hidden_dim_d = 350
n_layers_d = 2
G_var_threshold = 0.03
D_var_threshold = 0.03
scale_values = [(min_total[i], max_total[i]) for i in range(len(min_total))]
scale = 'minmax'

crnngan_curriculum = CRNNGAN(batch_length, sequence_length,
                 in_channels_g, out_channels_g, hidden_dim_g, n_layers_g,
                 in_channels_d, out_channels_d, hidden_dim_d, n_layers_d,
                 curriculum_learning = True, G_var_threshold=G_var_threshold,
                 D_var_threshold=D_var_threshold,
                 scale=scale, scale_values=scale_values)

cpu


In [5]:
crnngan_curriculum.trainloop(musicDataLoader)

Start training for 15 epochs
(100, 30, 4)
(100, 30, 4)
[0/15][0]	Loss_D: 6.7998	Loss_G: 5.8391	D(x): -3.3992	D(G(z)): -3.3997 / -0.00
 Time since epoch started: 5.07
There are a total of 0 fake songs saved already
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
[0/15][10]	Loss_D: 2.3394	Loss_G: 5.0299	D(x): -0.2930	D(G(z)): -0.2259 / -0.75
 Time since epoch started: 107.29
There are a total of 5 fake songs saved already
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)
(100, 30, 4)


KeyboardInterrupt: 